In [71]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

In [72]:
Q = 150  # m/s^3
A = 2000 # m^2
k = 400
kappa = k
R = 10000
L = 10000

pi = np.pi

a = A/pi

## Solving for $f(t)$

In this Notebook we use the coupling condition to solve for $f(t)$. We therefore need to solve the following equation:

$$ks_x(0) = \kappa \zeta_r(a)$$

$$-\frac{k}{L}f(t)  + k \sum_{n=0}^{\infty} T_n(t) \phi_n'(0) = \kappa \frac{1-f(t)}{R-a} + \kappa \sum_{n=0}^{\infty} T_n(t) \phi_n'(a)$$

where $T_n(t)$ is given as before:

We define:
$$ \phi(x) = \begin{bmatrix}
\phi_1(x) \\
\phi_2(x) \\
\vdots \\
\phi_N(x) \\ 
\end{bmatrix} $$

$$ T(t) = \begin{bmatrix}
T_1(t) \\
T_2(t) \\
\vdots \\
T_N(t) \\ 
\end{bmatrix} $$

$$ e^{-\lambda t} = \begin{bmatrix}
e^{-\lambda_1 t} & 0 & \cdots & 0\\
0 & e^{-\lambda_2 t} & \cdots & 0\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & e^{-\lambda_N t}\\ 
\end{bmatrix} $$

$$G = \begin{bmatrix}
<0,0> & <0,1> & \cdots & <0,N>\\
<1,0> & <1,1> & \cdots & <1,N>\\
\vdots & \vdots & \ddots & \vdots\\
<N,0> & <N,1> & \cdots & <N,N>\\ 
\end{bmatrix}$$
with $<n,m> = \int_0^L \phi_n(x) \phi_m(x) dx$ \\


$$ u_0 = \begin{bmatrix}
<u(x,0),1> \\
<u(x,0),2> \\
\vdots \\
<u(x,0),N> \\ 
\end{bmatrix} $$
with $<u(x,0),n> = \int_0^L u(x,0) \phi_n(x) dx$ \\


$$ q(t) = \int_0^L \begin{bmatrix}
L(x,t) & 0 & \cdots & 0\\
0 & L(x,t) & \cdots & 0\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & L(x,t)\\ 
\end{bmatrix}  \phi(x) dx$$
with $$ L(x,t) = -f'(t)\left(1-\frac{x}{L} \right) -  f(t)\frac{Q}{AL} $$

We can now write:
$$T(t) = e^{-\lambda t} \left( G^{-1} u_0 + \int_0^t e^{\lambda \tau} G^{-1} q(\tau) d\tau \right)$$

where
$$ \begin{aligned} & \int_0^t e^{\lambda \tau} G^{-1} q(\tau) d\tau \\
=& \int_0^t e^{\lambda \tau} G^{-1} \int_0^L \begin{bmatrix}
L(x,\tau) \phi_1(x)  \\
L(x,\tau) \phi_2(x) \\
\vdots \\
L(x,\tau) \phi_N(x) \\ 
\end{bmatrix}  dx d\tau \\
=& \int_0^L \int_0^t e^{\lambda \tau} G^{-1} \begin{bmatrix}
L(x,\tau) \phi_1(x)  \\
L(x,\tau) \phi_2(x) \\
\vdots \\
L(x,\tau) \phi_N(x) \\ 
\end{bmatrix} d\tau dx \\
=& \int_0^L \int_0^t e^{\lambda \tau} G^{-1} \left( -f'(\tau) (1-\frac{x}{L}) \phi(x) -f(\tau)\frac{Q}{AL} \phi(x) \right)d\tau dx \\
=& -\int_0^L \int_0^t e^{\lambda \tau} G^{-1} f'(\tau) (1-\frac{x}{L}) \phi(x) d\tau dx  
-\int_0^L \int_0^t e^{\lambda \tau} G^{-1} f(\tau)\frac{Q}{AL} \phi(x) d\tau dx \\ 
=& -\int_0^L G^{-1}\int_0^t e^{\lambda \tau} f'(\tau) d\tau (1-\frac{x}{L}) \phi(x)dx  
-\int_0^L G^{-1}\int_0^t e^{\lambda \tau} f(\tau) d\tau \frac{Q}{AL} \phi(x)dx \\ 
\end{aligned}$$

Now we use partial integration to remove $f'(\tau)$:
$$\begin{aligned} & \int_0^t e^{\lambda \tau} f'(\tau) d\tau \\
= & \left[ e^{\lambda \tau} f(\tau) \right]^t_0 - \int_0^t e^{\lambda \tau} \begin{bmatrix}
\lambda_1 & 0 & \cdots & 0\\
0 & \lambda_2 & \cdots & 0\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & \lambda_N\\ 
\end{bmatrix} f(\tau) d\tau
\end{aligned}$$

In [73]:
from scipy.special import jv
from scipy.special import yv
from scipy.optimize import root

In [92]:
def T_x(t_arr, f_arr):
    def Ig1(ti, fi):
        return epx_lx(-ti) @ lx * fi
    
    def S1(t_arr, f_arr):
        return epx_lx(-t_arr[-1]) * f_arr[-1] - f_arr[0]* np.diag(np.ones(N_x-1))
    
    
    def I1(t_arr, f_arr):
        res = np.zeros((N_x-1, N_x-1))
        for i in range(len(f_arr)-1):
            res += 0.5*(Ig1(t_arr[i], f_arr[i])+Ig1(t_arr[i+1], f_arr[i+1]))*dt
        return res
    
    func1 = lambda x: inv_x @ (S1(t_arr, f_arr) - I1(t_arr, f_arr)) @ ((1-x/L) * phix(x))
    
    T1 = -integrate.quad_vec(func1, 0, L)[0]
    
    def Ig2(ti, fi):
        return epx_lx(-ti) * fi
    
    def I2(t_arr, f_arr):
        res = np.zeros((N_x-1, N_x-1))
        for i in range(len(f_arr)-1):
            res += 0.5*(Ig2(t_arr[i], f_arr[i])+Ig2(t_arr[i+1], f_arr[i+1]))*dt
        return res    

    func2 = lambda x: inv_x @  I2(t_arr, f_arr) @ (Q/(A*L) * phix(x))
    
    T2 = -integrate.quad_vec(func2, 0, L)[0]
    
    def uf(x):
        return ss_x(x) - f_arr[-1]*(1-x/L)
    
    u0 = integrate.quad_vec(lambda x: uf(x)*phix(x), 0, L)[0]

    return epx_lx(t_arr[-1]) @ (inv_x @ u0 + T1 + T2) 


In [93]:
def T_r(t_arr, f_arr):
    def Ig1(ti, fi):
        return epx_lr(-ti) @ lr * fi
    
    def S1(t_arr, f_arr):
        return epx_lr(-t_arr[-1]) * f_arr[-1] - f_arr[0]* np.diag(np.ones(N_r-1))
    
    
    def I1(t_arr, f_arr):
        res = np.zeros((N_r-1, N_r-1))
        for i in range(len(f_arr)-1):
            res += 0.5*(Ig1(t_arr[i], f_arr[i])+Ig1(t_arr[i+1], f_arr[i+1]))*dt
        return res
    
    func1 = lambda r: inv_r @ (S1(t_arr, f_arr) - I1(t_arr, f_arr)) @ ((R-r)/(R-a) * phir(r))
    
    T1 = integrate.quad_vec(func1, a, R)[0]
    
    def Ig2(ti, fi):
        return epx_lr(-ti) * fi
    
    def I2(t_arr, f_arr):
        res = np.zeros((N_r-1, N_r-1))
        for i in range(len(f_arr)-1):
            res += 0.5*(Ig2(t_arr[i], f_arr[i])+Ig2(t_arr[i+1], f_arr[i+1])) *dt
        return res    

    func2 = lambda r: inv_r @  I2(t_arr, f_arr) @ ((kappa-Q/pi)/(r*(a-R)) * phir(r))
    
    T2 = integrate.quad_vec(func2, a, R)[0]
    
    def vf(r):
        return ss_r(r) - (f_arr[-1]-1)/(a-R)*r + (R*f_arr[-1]-a)/(R-a)
    
    v0 = integrate.quad_vec(lambda r: vf(r)*phir(r), a, R)[0]
    
    E3 = integrate.quad_vec(lambda r: t_arr[-1] * -1*(kappa-Q/pi)/(r*(a-R)) * inv_r @ phir(r), a, R)[0]

    return epx_lr(t_arr[-1]) @ (inv_r @ v0 + T1 + T2 + E3) 


[-6.45504993 -1.88339551 -1.47876268 -1.00759896 -0.84652893 -0.67899557
 -0.59556979 -0.51028074 -0.4595158  -0.40784955 -0.37376801 -0.33904917
 -0.31463533 -0.28961963 -0.27133583 -0.25236376 -0.23825814 -0.22326518
 -0.21221352 -0.1999122  -0.19130483 -0.18077227 -0.17446399 -0.16480205
 -0.16151221 -0.15100642 -0.15525569 -0.13447488 -0.20465433]
[ 7.99934835 -2.10416649  2.00347998 -0.35793602 -0.36456223  0.63774727
 -1.76868761  1.21859082 -2.51081411  1.45570451 -2.6891862   1.39537892
 -2.39669411  1.09735384 -1.7555763   0.63610337 -0.91388834  0.0488323 ]


In [116]:
f_arr = np.array([ss_x(0)])

Tm = 1e-10
dt = Tm/10
t_arr = np.array([0])

def tot(t_arr, f_arr):
    x_term = lambda f_arr: -k*f_arr[-1]/L + k* T_x(t_arr, f_arr) @ phix_prime(0)
    r_term = lambda f_arr: kappa*(f_arr[-1]-1)/(a-R) + kappa* T_r(t_arr, f_arr) @ phir_prime(a)
    return x_term(f_arr) - r_term(f_arr)

for i in range(1,int(Tm/dt)):
    print(i)
    t_arr = np.append(t_arr, np.array([t_arr[-1]+dt]))
    func = lambda fnew: tot(t_arr, np.append(f_arr, np.array([fnew])))
    fi = root(func, x0 = 2*f_arr[-1], method='lm').x
    f_arr = np.append(f_arr, np.array([fi]))
    
print(t_arr)
print(f_arr)

1
2
3
4
5
6
7
8
9
[0.e+00 1.e-11 2.e-11 3.e-11 4.e-11 5.e-11 6.e-11 7.e-11 8.e-11 9.e-11]
[0.51814035 0.31319245 0.31319245 0.31319245 0.31319245 0.31319245
 0.31319245 0.31319245 0.31319245 0.31319245]


In [115]:
def tot2(t_arr, f_arr):
    x_term = lambda f_arr: -k*f_arr[-1]/L + k* T_x(t_arr, f_arr) @ phix_prime(0)
    r_term = lambda f_arr: kappa*(f_arr[-1]-1)/(a-R) + kappa* T_r(t_arr, f_arr) @ phir_prime(a)
    return [x_term(f_arr) - r_term(f_arr)]*20
#, x_term(f_arr) - r_term(f_arr), x_term(f_arr) - r_term(f_arr)


tm = 1e-6
N = 20
dt = tm/N
t_arr = np.linspace(0,tm,21)
func2 = lambda f_arr: tot2(t_arr,np.append(np.array([ss_x(0)]), f_arr))
print(root(func2, x0=[ss_x(0)]*10 + [ssn_x(0)]*10, method='lm').x)

[0.51814035 0.51814035 0.51814035 0.51814035 0.51814035 0.51814035
 0.51814035 0.51814035 0.51814035 0.51814035 0.71981966 0.71981966
 0.71981966 0.71981969 0.71981966 0.71981966 0.71981966 0.71981966
 0.71981966 0.31319245]


In [78]:
# A = phix_prime(0)
# B = T_x(t_arr, f_arr)
# print(A)
# print(B)
# print(A @ B)

In [83]:
print(ss_x(0))
print(ssn_x(0))

0.5181403493129045
0.7198196644388819


### Finding eigenfunctions and -values

In [6]:
order = Q/(2*kappa*pi)
def phi_bessel(r,par):
    return (r)**(order)*jv(order, np.sqrt(par[0]/kappa)*(r)) + par[1]*(r)**(order)*yv(order, np.sqrt(par[0]/kappa)*(r))


In [7]:
good_roots_bessel = np.array([[3.94030436e-05,7.61848195e-01], #n=0
                              [1.71476791e-04,1.36154281e+00], #n=1
                              [3.94842613e-04,2.49685017e+00], #n=2
                              [7.08883157e-04,6.97900847e+00], #n=3
                              [1.11333787e-03,-1.12220582e+01], #n=4
                              [1.60807412e-03,-3.06309775e+00], #n=5
                              [2.19302e-03,-1.67100131e+00], #n=6
                              [2.86812e-03,-1.05017646e+00], #n=7
                              [3.63335e-03,-0.66912985e+00], #n=8
                              [4.4887e-03,-0.38862351e+00], #n=9
                              [5.43415e-03,-0.15366476e+00], #n=10
                              [6.46968e-03,0.065137e+00], #n=11
                              [7.5953e-03,0.2897497e+00], #n=12
                              [8.81099e-03,0.54445193e+00], #n=13
                              [1.011676e-02,0.86784664e+00], #n=14
                              [1.15126e-02,1.34271697e+00], #n=15
                              [1.29985e-02,2.21252377e+00], #n=16
                              [1.457447e-02,4.71344792e+00], #n=17
                              [1.62405098e-02,-2.61426615e+02], #n=18
                              [1.799661e-02,-4.44488786e+00]]) #n=20


In [8]:
N_r = 19 #len(good_roots_bessel)
N_x = 30

def phi_r_n(r,n):
    return phi_bessel(r, good_roots_bessel[n-1])

def labda_r_n(n):
    return good_roots_bessel[n-1][0]

def phi_x_n(x,n):
    return np.exp(-Q/(2*k*A)*x)*np.sin(n*pi*x/L)

def labda_x_n(n):
    return (Q/A)**2/(4*k) + k*(n*pi/L)**2

def phi_x_prime(x,n):
    return n*pi/L

def phi_r_prime(r,n):
    c = order
    w = np.sqrt(good_roots_bessel[n-1][0]/kappa)
    d = good_roots_bessel[n-1][1]
    
    T1 = d*w*r*yv(c-1, w*r)
    T2 = 2*c*d*yv(c, w*r)
    T3 = -d*w*r*yv(c+1,w*r)
    T4 = w*r*jv(c-1,w*r)
    T5 = 2*c*jv(c,w*r)
    T6 = -w*r*jv(c+1,w*r)
    return 0.5*r**(c-1)*(T1+T2+T3+T5+T6)

In [9]:
def psi(r,t):
    return (f_interp(t)-1)/(a-R)*r + (R*f_interp(t)-a)/(R-a)

def xi(x,t):
    return f_interp(t)*(1-x/L)

def ss_x(x):
    s0 = (a/R)**(2*Q/(kappa*pi))
    v = -2*Q/(A*k)
    return s0/(1-np.exp(v*L))*np.exp(v*x) + s0 - s0/(1-np.exp(v*L))


def ss_r(r):
    #return (1-(a/R)**(0.5*Q/(kappa*pi)))/(R-a)*r + 1-R*(1-(a/R)**(0.5*Q/(kappa*pi)))/(R-a)
    return (r/R)**(2*Q/(kappa*pi))

def ssn_x(x):
    s0 = (a/R)**(Q/(kappa*pi))
    v = -Q/(A*k)
    return s0/(1-np.exp(v*L))*np.exp(v*x) + s0 - s0/(1-np.exp(v*L))

def ssn_r(r):
    #return (1-(a/R)**(0.5*Q/(kappa*pi)))/(R-a)*r + 1-R*(1-(a/R)**(0.5*Q/(kappa*pi)))/(R-a)
    return (r/R)**(Q/(kappa*pi))

def u(x):
    return ss_x(x) - xi(x,0)

def v(r):
    return ss_r(r) - psi(r,0)


In [58]:
def H(r,t):
    psi_t = f_prime_interp(t)*r/(a-R) + R*f_prime_interp(t)/(R-a)
    psi_rr = 0
    psi_r = (f_interp(t)-1)/(a-R)
    return np.diag((-psi_t + kappa*psi_rr + (kappa-Q/pi)/r*psi_r)*np.ones(N_r-1))

def Lf(x,t):
    xi_t = f_prime_interp(t)*(1-x/L)
    xi_xx = 0
    xi_x = -f_interp(t)/L
    return np.diag((-xi_t + k*xi_xx + Q/A*xi_x )*np.ones(N_x-1))

def inner_x(n,m):
    return integrate.quad(lambda x: phi_x_n(x,m)*phi_x_n(x,n), 0, L)[0]

def inner_r(n,m):
    return integrate.quad(lambda r: phi_r_n(r,m)*phi_r_n(r,n), a, R)[0]

G_x = np.array([[inner_x(n,m) for n in range(1,N_x)] for m in range(1,N_x)])
G_r = np.array([[inner_r(n,m) for n in range(1,N_r)] for m in range(1,N_r)])

labda_x = np.array([labda_x_n(n) for n in range(1,N_x)])
labda_r = np.array([labda_r_n(n) for n in range(1,N_r)])
lx = np.diag(labda_x)
lr = np.diag(labda_r)
def epx_lx(t):
    return np.diag(np.exp(-t*labda_x))
def epx_lr(t):
    return np.diag(np.exp(-t*labda_r))

inv_x = np.linalg.inv(G_x)
inv_r = np.linalg.inv(G_r)


def phix(x):
    return np.array([phi_x_n(x,n) for n in range(1,N_x)])

def phir(r):
    return np.array([phi_r_n(r,n) for n in range(1,N_r)])


def phix_prime(x):
    return np.array([phi_x_prime(x,n) for n in range(1,N_x)])

def phir_prime(r):
    return np.array([phi_r_prime(r,n) for n in range(1,N_r)])

# def T_x(t):
#     I = lambda tau: integrate.quad_vec(lambda x: Lf(x,tau) @ phix, 0,L)[0]
#     func = lambda tau: exp_lx @ inv_x @ I(tau)
#     return integrate.quad_vec(func, 0,t)[0]

## Another attempt: Galerkin-like approach

We assume $$f(t) = a_0 + \sum_{m=1}^M a_m e^{-\mu_m t}$$

In [128]:
Tmax = 1e2

M = 10
Px = np.zeros((N_x-1,M))
Pr = np.zeros((N_x-1,M))


mu = np.append(np.array([0]),np.logspace(-6,1,M-1))

def func(i):
    
    def U0(t):
        return inv_x @ integrate.quad_vec(lambda x: (ss_x(x)-np.exp(-mu[i]*t))*(1-x/L) * phix(x), 0,L)[0]
    
    def S1(t):
        temp = lambda x: ( ss_x(x) - np.exp(-mu[i]*t)*(1-x/L) ) *phix(x)
        return inv_x @ integrate.quad_vec(temp, 0,L)[0] 
  
    def S2(t):
        I1 = integrate.quad_vec(lambda tau: epx_lx(tau)*mu[i]*np.exp(-mu[i]*tau) @ np.ones(N_x-1), 0,t)[0]
        I2 = integrate.quad_vec(lambda x: inv_x @ ( (1-x/L)*phix(x) ), 0,L)[0]
        return I1 * I2 
    
    def S3(t):
        I1 = integrate.quad_vec(lambda tau: epx_lx(tau)*-Q/(A*L)*np.exp(-mu[i]*tau) @ np.ones(N_x-1), 0,t)[0]
        I2 = integrate.quad_vec(lambda x: inv_x @ phix(x), 0,L)[0]
        return I1 * I2 
    
    temp = lambda t: epx_lx(-t) @ (S1(t) + S2(t) + S3(t))
    return integrate.quad_vec(lambda t: temp(t)*np.exp(-mu[i]*t), 0, Tmax)[0]


t = Tmax
for i in range(M):
    Px[:,i] = func(i)
        
print(Px)

[[-7.76528648e+01 -7.76394868e+01 -7.75525964e+01 -7.69039043e+01
  -7.21985051e+01 -4.45006848e+01  3.22943409e+00  9.72359708e-01
   1.29630363e-01  1.72858485e-02]
 [-1.51652462e+01 -1.51615746e+01 -1.51377282e+01 -1.49597551e+01
  -1.36718207e+01 -6.23011767e+00  4.28950843e+00  7.78850391e-01
   1.03730385e-01  1.38303423e-02]
 [-1.28021407e+01 -1.27991971e+01 -1.27800788e+01 -1.26373878e+01
  -1.16044694e+01 -5.62199016e+00  3.07435529e+00  5.65694688e-01
   7.52407704e-02  1.00300437e-02]
 [-7.91140277e+00 -7.90939934e+00 -7.89638761e+00 -7.79928073e+00
  -7.09675144e+00 -3.04688535e+00  2.52317323e+00  4.44801117e-01
   5.90377137e-02  7.86789155e-03]
 [-7.01168104e+00 -7.00995403e+00 -6.99873760e+00 -6.91502750e+00
  -6.30933558e+00 -2.81387422e+00  2.05888455e+00  3.61672922e-01
   4.78846663e-02  6.37942366e-03]
 [-5.49320628e+00 -5.49179699e+00 -5.48264405e+00 -5.41433660e+00
  -4.92022436e+00 -2.07480529e+00  1.79194170e+00  3.06397087e-01
   4.04349918e-02  5.38462682e-03